In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
movie = pd.read_csv("tmdb_5000_movies.csv")
credit = pd.read_csv("tmdb_5000_credits.csv")

In [3]:
movie.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [29]:
credit.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [30]:
#change movie_id to id to reviews
credit = credit.rename(columns={'movie_id': 'id'})

In [31]:
credit.head()

,id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [40]:
credit.column = ['id','title','cast','crew']
movie = movie.merge(credit,on = 'id')

In [41]:
movie['overview'].head(5)

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

In [42]:
movie['overview'] = movie['overview'].fillna('')

In [43]:
def create_soup(x):
    return ''.join(x['keywords']) + '' + ''.join(x['genres']) + '' + ''.join(x['overview'])
movie['soup'] = movie.apply(create_soup, axis = 1)

In [44]:
tfidf = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = tfidf.fit_transform(movie['soup'])
tfidf_matrix.shape

(4803, 32768)

In [45]:
similarity_matrix = linear_kernel(tfidf_matrix, tfidf_matrix)
similarity_matrix

array([[1.        , 0.17829479, 0.10885591, ..., 0.08473419, 0.00270529,
        0.04361802],
       [0.17829479, 1.        , 0.10261117, ..., 0.08387287, 0.        ,
        0.0399101 ],
       [0.10885591, 0.10261117, 1.        , ..., 0.05355364, 0.        ,
        0.02475109],
       ...,
       [0.08473419, 0.08387287, 0.05355364, ..., 1.        , 0.01657892,
        0.03835153],
       [0.00270529, 0.        , 0.        , ..., 0.01657892, 1.        ,
        0.01014256],
       [0.04361802, 0.0399101 , 0.02475109, ..., 0.03835153, 0.01014256,
        1.        ]])

In [46]:
# construct a reverse map of idices of movies title
indices = pd.Series(movie.index, index = movie['title']).drop_duplicates()
indices

title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

In [47]:
def get_recommendations(title, similarity_matrix=similarity_matrix):
    #get the index of the movie that matches the title
    idy = indices[title]

    #similarity_score is the list of index and similarity matrix or get the pairwise similarity scores of all movies with tht movie
    similarity_score = list(enumerate(similarity_matrix[idy]))

    #sort in descending order the similarity score of movie inputted with all the other movies or sort movies base on similarity score.
    similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)

    # Get the scores of the 12 most similar movies. Ignore the first movie.
    similarity_score = similarity_score[1:13]

    #return movie names using the mapping series or get the movie indices
    write a detailed project on a recommendation system using a movie recommendations-based system with the following code and explain it to beginners with a detailed explanation: 


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel   

movie = pd.read_csv("tmdb_5000_movies.csv")
credit = pd.read_csv("tmdb_5000_credits.csv")

#change movie_id to id to reviews
credit = credit.rename(columns={'movie_id': 'id'})

credit.column = ['id','title','cast','crew']
movie = movie.merge(credit,on = 'id')

credit.column = ['id','title','cast','crew']
movie = movie.merge(credit,on = 'id')

movie['overview'] = movie['overview'].fillna('')

def create_soup(x):
    return ''.join(x['keywords']) + '' + ''.join(x['genres']) + '' + ''.join(x['overview'])
movie['soup'] = movie.apply(create_soup, axis = 1)

tfidf = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = tfidf.fit_transform(movie['soup'])

similarity_matrix = linear_kernel(tfidf_matrix, tfidf_matrix)

# construct a reverse map of idices of movies title
indices = pd.Series(movie.index, index = movie['title']).drop_duplicates()

def get_recommendations(title, similarity_matrix=similarity_matrix):
    #get the index of the movie that matches the title
    idy = indices[title]

    #similarity_score is the list of index and similarity matrix or get the pairwise similarity scores of all movies with tht movie
    similarity_score = list(enumerate(similarity_matrix[idy]))

    #sort in descending order the similarity score of movie inputted with all the other movies or sort movies base on similarity score.
    similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)

    # Get the scores of the 12 most similar movies. Ignore the first movie.
    similarity_score = similarity_score[1:13]

    #return movie names using the mapping series or get the movie indices
    movie_indices = [i[0] for i in similarity_score]

    #return the top 12 most similar movies

    return (movie['title'].iloc[movie_indices])

get_recommendations('The Dark Knight Rises', similarity_matrix)


    #return the top 12 most similar movies

    return (movie['title'].iloc[movie_indices])

In [49]:
get_recommendations('The Dark Knight Rises', similarity_matrix)

65                              The Dark Knight
428                              Batman Returns
119                               Batman Begins
1359                                     Batman
299                              Batman Forever
210                              Batman & Robin
9            Batman v Superman: Dawn of Justice
2428                          Brooklyn's Finest
3854    Batman: The Dark Knight Returns, Part 2
2442                            Southland Tales
606                                    Blackhat
2217                   Everyone Says I Love You
Name: title, dtype: object